In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_dataset = datasets.FashionMNIST(root='mnist',download=False,train=True,transform=ToTensor())
testing_dataset = datasets.FashionMNIST(root='mnist',download=False,train=False,transform=ToTensor())

In [3]:
batch_size = 64

In [4]:
train_loader = DataLoader(training_dataset,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(testing_dataset,batch_size=batch_size,shuffle=True)

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(nn.Linear(28*28,512),
                                               nn.ReLU(),
                                               nn.Linear(512,512),
                                               nn.ReLU(),
                                               nn.Linear(512,10))
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
model = NeuralNetwork()

## Tuning hyperparameters

In [7]:
learning_rate = 1e-3
epochs = 5

In [8]:
loss_fn = nn.CrossEntropyLoss()

In [9]:
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [10]:
def train_loop(dataloader,model,optimizer,loss_fn):
    size = len(dataloader.dataset)
    model.train()
    for count,(X,y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred,y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if count % 100 == 0:
            print(f"Loss - {loss.item()}, {count * batch_size + len(X)} / {size}")

In [11]:
def test_loop(dataloader,model,loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss,correct = 0,0

    with torch.no_grad():
        for X,y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Accuracy - {100*correct}, loss - {test_loss}")

In [12]:
epochs = 10
for t in range(epochs):
    train_loop(train_loader,model,optimizer,loss_fn)
    test_loop(test_loader,model,loss_fn)

Loss - 2.301722526550293, 64 / 60000
Loss - 0.6619126200675964, 6464 / 60000
Loss - 0.4067089855670929, 12864 / 60000
Loss - 0.5594701170921326, 19264 / 60000
Loss - 0.6906286478042603, 25664 / 60000
Loss - 0.3670739531517029, 32064 / 60000
Loss - 0.40212056040763855, 38464 / 60000
Loss - 0.4724532663822174, 44864 / 60000
Loss - 0.44237464666366577, 51264 / 60000
Loss - 0.5012404322624207, 57664 / 60000
Accuracy - 84.11999999999999, loss - 0.4287745150601029
Loss - 0.35242873430252075, 64 / 60000
Loss - 0.38580578565597534, 6464 / 60000
Loss - 0.41649654507637024, 12864 / 60000
Loss - 0.36890920996665955, 19264 / 60000
Loss - 0.32571202516555786, 25664 / 60000
Loss - 0.31311240792274475, 32064 / 60000
Loss - 0.3588408827781677, 38464 / 60000
Loss - 0.5785338282585144, 44864 / 60000
Loss - 0.2744130492210388, 51264 / 60000
Loss - 0.4797165095806122, 57664 / 60000
Accuracy - 85.86, loss - 0.3774876029817921
Loss - 0.3996089696884155, 64 / 60000
Loss - 0.24483925104141235, 6464 / 60000
Lo

In [15]:
torch.save(model.state_dict(),'model1.pth')